# Исследование надежности заемщиков


Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

#### Данные имеют следующие столбцы

- children — количество детей в семье
- days_employed — общий трудовой стаж в днях
- dob_years — возраст клиента в годах
- education — уровень образования клиента
- education_id — идентификатор уровня образования
- family_status — семейное положение
- family_status_id — идентификатор семейного положения
- gender — пол клиента
- income_type — тип занятости
- debt — имел ли задолженность по возврату кредитов
- total_income — ежемесячный доход
- purpose — цель получения кредита

## Откроем таблицу и изучим общую информацию о данных

In [ ]:
import pandas as pd
import numpy as np
try:
    data = pd.read_csv('/datasets/data.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')

In [2]:
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Предобработка данных

### Удаление пропусков

In [4]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

### Заполнение пропусков

In [5]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

### Обработка аномальных значений

In [6]:
data['days_employed'] = data['days_employed'].abs()

**Для каждого типа занятости выведем медианное значение трудового стажа `days_employed` в днях.**

In [7]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получатся аномально большие значения. Исправить такие значения сложно, поэтому оставим их как есть. Тем более этот столбец не понадобится для дальнейшего исследования.

**Выведем перечень уникальных значений столбца `children`.**

In [8]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

**В столбце `children` есть два аномальных значения. Удалим строки, в которых встречаются такие аномальные значения из датафрейма `data`.**

In [9]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

**Ещё раз выведем перечень уникальных значений столбца `children`, чтобы убедиться, что артефакты удалены.**

In [10]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5])

**Заполним пропуски в столбце `days_employed` медианными значениями по каждому типу занятости `income_type`.**

In [11]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

**Убедимся, что все пропуски заполнены.**

In [12]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Изменение типов данных

In [13]:
data['total_income'] = data['total_income'].astype(int)

### Обработка дубликатов

In [14]:
data['education'] = data['education'].str.lower()

In [15]:
data.duplicated().sum()

71

In [16]:
data = data.drop_duplicates()

### Категоризация данных

**Задание 16. На основании диапазонов, указанных ниже, создадим в датафрейме `data` столбец `total_income_category` с категориями:**

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.


**Например, кредитополучателю с доходом 25000 нужно назначить категорию `'E'`, а клиенту, получающему 235000, — `'B'`. Используем собственную функцию с именем `categorize_income()` и метод `apply()`.**

In [17]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [18]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

**Выведем на экран перечень уникальных целей взятия кредита из столбца `purpose`.**

In [19]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

**Создадим функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:**

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.

**Например, если в столбце `purpose` находится подстрока `'на покупку автомобиля'`, то в столбце `purpose_category` должна появиться строка `'операции с автомобилем'`.**

**Используем собственную функцию с именем `categorize_purpose()` и метод `apply()`. Изучим данные в столбце `purpose` и определим, какие подстроки помогут правильно определить категорию.**

In [20]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [21]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

###  Исследуем данные и ответим на поставленные вопросы иследования

#### Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [22]:
grouped_children = pd.DataFrame() #создаём отдельный датафрейм для работы над новыми ячейками, связанными с количеством деетй у заемщиков
grouped_children['count_children'] = data.groupby('children')['debt'].count() #группируем датафрейм по количеству детей и считаем сколько заемщиков по количеству детей 
grouped_children['sum_children'] = data.groupby('children')['debt'].sum() #группируем датафрейм по количеству детей и считаем сколько просрочек по кредитам в каждой категории
grouped_children['result_children'] = grouped_children['sum_children'] / grouped_children['count_children'] #находим отношение количества просрочек к количеству заемщиков с детьми, чтобы посчитать "коэффициент просрочки"
grouped_children['result_children'] = list(map(lambda cell: "{:.2%}".format(cell), grouped_children['result_children'])) #настраиваем формат отображения столбца 'result_children' в процентах и двумя знаками после запятой, так более наглядно и понятно
grouped_children.sort_values('result_children', ascending = False) #сортируем заемщиков с данным количеством детей по просрочке

,count_children,sum_children,result_children
children,,,
4,41,4,9.76%
2,2052,194,9.45%
1,4808,444,9.23%
3,330,27,8.18%
0,14091,1063,7.54%
5,9,0,0.00%


**Вывод:** С увеличением количества детей у заемщика мы видим увеличение количества просроченных задолженностей, хотя люди с 3 детьми чаще платят в срок чем люди с 1 и 2 детьми. Данные неоднозначные, возможно, нужна большая выборка, чем та, которую мы имеем. По крайней мере, это точно можно сказать для заемщиков с 5 детьми, их слишком мало в выборке, чтобы делать хоть сколько-нибудь обоснованные выводы. Бездетные заемщики, как правило реже просрачивают оплату по кредиту, чем люди с детьми.

#### Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [23]:
# Ваш код будет здесь. Вы можете создавать новые ячейки.
debt_family_status = pd.DataFrame() #создаём отдельный датафрейм для работы над новыми ячейками, связанными с семейным статусом заемщиков
debt_family_status['count_family_status'] = data.groupby('family_status')['debt'].count() #группируем датафрейм по семейному статусу и считаем сколько заемщиков по этой категории
debt_family_status['sum_family_status'] = data.groupby('family_status')['debt'].sum() #группируем датафрейм по семейному статусу и считаем сколько просрочек по кредитам в каждой категории
debt_family_status['result_family_status'] = debt_family_status['sum_family_status'] / debt_family_status['count_family_status'] #находим отношение количества просрочек к количеству заемщиков с данным семейным статусом, чтобы посчитать "коэффициент просрочки"
debt_family_status['result_family_status'] = list(map(lambda cell: "{:.2%}".format(cell), debt_family_status['result_family_status'])) #настраиваем формат отображения столбца 'result_family_status' в процентах и двумя знаками после запятой, так более наглядно и понятно
debt_family_status.sort_values('result_family_status', ascending = False) #сортируем заемщиков с данным семейным статусом по просрочке

,count_family_status,sum_family_status,result_family_status
family_status,,,
Не женат / не замужем,2796,273,9.76%
гражданский брак,4134,385,9.31%
женат / замужем,12261,927,7.56%
в разводе,1189,84,7.06%
вдовец / вдова,951,63,6.62%


**Вывод:**  Имеется зависимость между семейным положением и возвратом кредита в срок. Люди, несостоящие в браке, чаще не возвращают кредит в срок. Однако, те заемщики, которые  развелись или овдовели чаще возвращают деньги в срок, чем люди, состоящие в браке.

#### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [24]:
# Ваш код будет здесь. Вы можете создавать новые ячейки.
debt_total_income = pd.DataFrame() #создаём отдельный датафрейм для работы над новыми ячейками, связанными с семейным статусом заемщиков
debt_total_income['count'] = data.groupby('total_income_category')['debt'].count() #группируем датафрейм по уровню дохода и считаем сколько заемщиков по этой категории
debt_total_income['sum'] = data.groupby('total_income_category')['debt'].sum() #группируем датафрейм по уровню дохода и считаем сколько просрочек по кредитам в каждой категории
debt_total_income['result_total_income'] = debt_total_income['sum'] / debt_total_income['count'] #находим отношение количества просрочек к количеству заемщиков с данным увровнем дохода, чтобы посчитать "коэффициент просрочки"
debt_total_income['result_total_income'] = list(map(lambda cell: "{:.2%}".format(cell), debt_total_income['result_total_income'])) #настраиваем формат отображения столбца 'result_total_income' в процентах и двумя знаками после запятой, так более наглядно и понятно
debt_total_income.sort_values('result_total_income', ascending = False) #сортируем заемщиков с данным уровнем дохода по просрочке

,count,sum,result_total_income
total_income_category,,,
E,22,2,9.09%
C,15921,1353,8.50%
A,25,2,8.00%
B,5014,354,7.06%
D,349,21,6.02%


**Вывод:** Зависимость между уровнем дохода заемщика и возвратом кредита в срок отсутствует (хотя этот вывод и контринтуитивен, исследование показала такую картину). Больше всего просрочек по кредитам в категории

In [25]:
data_total_income_new = data.groupby('total_income_category').agg({'debt': ['count', 'sum', 'mean']}).sort_values(by = ('debt', 'mean'), ascending=False)
def percent(row):
    return "{0:.2%}".format(row)
data_total_income_new['debt','mean'] = data_total_income_new['debt','mean'].apply(percent)
data_total_income_new

debt             
                       count   sum   mean
total_income_category                    
E                         22     2  9.09%
C                      15921  1353  8.50%
A                         25     2  8.00%
B                       5014   354  7.06%
D                        349    21  6.02%

#### Как разные цели кредита влияют на его возврат в срок?

In [29]:
debt_purpose_category_new = data.pivot_table(index='purpose_category', values='debt', aggfunc=('count', 'sum', 'mean'))
debt_purpose_category_new['mean'] = debt_purpose_category_new['mean'].apply(percent)
debt_purpose_category_new

,count,mean,sum
purpose_category,,,
операции с автомобилем,4279.0,9.35%,400.0
операции с недвижимостью,10751.0,7.26%,780.0
получение образования,3988.0,9.25%,369.0
проведение свадьбы,2313.0,7.91%,183.0


**Вывод:** Из данных видно, что среди всех категорий, чаще всего платят в срок с целью кредита на недвижимость, чаще просрочка у тех кому нужен кредит на операции с автомобилем и образованием. Кредиты на свадьбу просрачивают реже - примерно как и в случае с просрочками по недвижимости.

In [30]:
def percent(row):
    return "{0:.2%}".format(np.mean(row))

In [31]:
debt_purpose_category_new = data.pivot_table(index='purpose_category', values='debt', aggfunc=['count', 'sum', percent])
debt_purpose_category_new

,count,sum,percent
,debt,debt,debt
purpose_category,,,
операции с автомобилем,4279,400,9.35%
операции с недвижимостью,10751,780,7.26%
получение образования,3988,369,9.25%
проведение свадьбы,2313,183,7.91%


### Общий вывод.

Для построения модели кредитного скоринга придётся учитывать различные данные, связанные с заемщиком:
* **Количество детей**. *(Чем больше детей у заемщика, тем больше вероятность просрочки (не включая в эту выборку людей с 5 детьми, нужно больше данных по этой категории заемщиков)*


* **Семейное положение**. *(Люди, состоящие в браке чаще возвращают деньги в срок, чем несостоящие в браке, а овдовевшие или разведенные, возвращают чаще, чем люди состоящие в браке.)*


* **Цель кредита**. *(Вероятность просрочки ниже для кредитов на операции с недвижимостью, и выше для целей образования и операций с автомобилем, кредиты на свадьбу возвращают в срок почти также часто как кредиты на операции с недвижимостью.)*

Как ни странно, в результате исследования выяснилось, что **доход заемщика не влияет на вероятноть просрочки**

In [32]:
debt_purpose_category_new = data.pivot_table(index='purpose_category', columns='children', values='debt', aggfunc=['count', 'sum', percent])
debt_purpose_category_new

count                          sum                  \
children                     0     1     2    3   4  5    0    1   2   3  4   
purpose_category                                                              
операции с автомобилем    2845   963   399   60  10  2  243  103  48   5  1   
операции с недвижимостью  7074  2447  1035  169  21  5  476  200  88  13  3   
получение образования     2642   866   403   69   7  1  229   90  46   4  0   
проведение свадьбы        1530   532   215   32   3  1  115   51  12   5  0   

                            percent                                         
children                  5       0       1       2       3       4      5  
purpose_category                                                            
операции с автомобилем    0   8.54%  10.70%  12.03%   8.33%  10.00%  0.00%  
операции с недвижимостью  0   6.73%   8.17%   8.50%   7.69%  14.29%  0.00%  
получение образования     0   8.67%  10.39%  11.41%   5.80%   0.00%  0.00%  
проведение свадьбы        0   7.52%   9.59%   5.58%  15.62%   0.00%  0.00%

In [33]:
data.groupby(['purpose_category', 'children'])['debt'].agg(['count', 'sum', percent])

count  sum percent
purpose_category         children                    
операции с автомобилем   0          2845  243   8.54%
                         1           963  103  10.70%
                         2           399   48  12.03%
                         3            60    5   8.33%
                         4            10    1  10.00%
                         5             2    0   0.00%
операции с недвижимостью 0          7074  476   6.73%
                         1          2447  200   8.17%
                         2          1035   88   8.50%
                         3           169   13   7.69%
                         4            21    3  14.29%
                         5             5    0   0.00%
получение образования    0          2642  229   8.67%
                         1           866   90  10.39%
                         2           403   46  11.41%
                         3            69    4   5.80%
                         4             7    0   0.00%
                         5             1    0   0.00%
проведение свадьбы       0          1530  115   7.52%
                         1           532   51   9.59%
                         2           215   12   5.58%
                         3            32    5  15.62%
                         4             3    0   0.00%
                         5             1    0   0.00%

# Итоговый вывод
Проведенная работа над исследованием надёжности заёмщиков завершена.

На основе имеющихся таблиц вывести следущие закономерности:

- Чем выше и лучше твое образование, тем больше ответсвенности и понимания приходит по поводу кредита. 
- С возрастом приходит понимание как распределять деньги и возвращать кредит вовремя.
- Наличие детей в целом негативно влияет на возврат кредита.
- Если люди живут только ради себя, вероятность невозврата кредита увеличивается.
- Люди с высоким доходом успешнее возвращают кредит, почти так же, как и люди с низким доходом.
- Цель кредита, повышающий качество жизни без серьезных расходов - повышает возврат кредита.

Опишем идеального кандидата на возврат кредита по данным этой выборки без учета возраста и образования:
Человек, потерявщий супруга, не имеет детей, с доходом от 320 тысяч, который делает операции с жилой недвижимостью. Человек, обладающий опытом жизни и понимающий, что он может расчитывать только на себя и при его скромных доходах он эффективно планирует возврат кредита.

Кандидат с наименьшой вероятностью по выдачи кредита:
Человек, живущий один, с доходом от 120 до 200 тысяч желает купить машину в кредит.

Проект состоял из следущих задач, которые успешно выполнены:

1. Оценка аномалий и определение их типа.
2. Выявление пропусков и их заполнение, замена некорректных значений.
3. Приведение дублирующих названий в один вид и удаление полных дупликатов.
4. Лемматизация целей дохода и выявление категорий.
5. Проведение сводного анализа для подтверждение гипотез.
6. Получение опыта в распределении доходов по миру